In [1]:
from platform import python_version
# !pip install shap==0.39
# !pip install transformers==4.5
python_version()

'3.7.13'

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline
from data.loader import dataset_loader, DatasetEnum
import shap

# Load model from huggingface

In [4]:
tokenizer = AutoTokenizer.from_pretrained("ghanashyamvtatti/roberta-fake-news")
model = AutoModelForSequenceClassification.from_pretrained("ghanashyamvtatti/roberta-fake-news").cpu()
pipeline = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True)

# Load the respective dataset for the model

In [5]:
true_dataset, fake_dataset = dataset_loader(DatasetEnum.ROBERTA_FAKE_NEWS)

Loading true instances from dir: /home/sersery/Desktop/TUM Informatik/SS22/Thesis/Code/Explainability_of_FND_Models/Huggingface/datasets/RobertaFakeNews/True.csv
Loading fake instances from dir: /home/sersery/Desktop/TUM Informatik/SS22/Thesis/Code/Explainability_of_FND_Models/Huggingface/datasets/RobertaFakeNews/Fake.csv


In [6]:
def explain_text(text):
    print(f'Explaining the following text: \n---------------------------------------------------------\n{text}\n--------------------------------------------------------')
    prediction = pipeline([text])
    print(f'Prediction: {prediction}')
    explainer = shap.Explainer(pipeline)
    shap_values = explainer([text])
    shap.text_plot(shap_values)

In [11]:
sample = true_dataset["text"][0][:512]
explain_text(sample)

Explaining the following text: 
---------------------------------------------------------
WASHINGTON (Reuters) - The head of a conservative Republican faction in the U.S. Congress, who voted this month for a huge expansion of the national debt to pay for tax cuts, called himself a “fiscal conservative” on Sunday and urged budget restraint in 2018. In keeping with a sharp pivot under way among Republicans, U.S. Representative Mark Meadows, speaking on CBS’ “Face the Nation,” drew a hard line on federal spending, which lawmakers are bracing to do battle over in January. When they return from the h
--------------------------------------------------------
Prediction: [[{'label': 'LABEL_0', 'score': 9.815104021981824e-06}, {'label': 'LABEL_1', 'score': 0.9999902248382568}]]


  0%|          | 0/48 [00:00<?, ?it/s]

,,WASHINGTON,(,Reuters,),- The head of,a conservative Republican faction,in the U.,"S. Congress, who voted this month for a huge expansion of the national debt to pay for tax cuts,",called himself a ““fiscal conservative”” on Sunday and urged budget restraint in 2018.,"In keeping with a sharp pivot under way among Republicans, U.S.","Representative Mark Meadows, speaking on CBS’’ ““Face the Nation,”” drew a hard line on federal spending, which lawmakers are bracing to do battle over in January. When they return from the h"
LABEL_0,-0.025,-1.637,-0.794,-2.227,-0.366,-3.246,-0.964,-0.477,-0.418,-1.321,-0.601,-0.486
LABEL_1,0.025,1.637,0.793,2.226,0.366,3.246,0.964,0.477,0.416,1.32,0.602,0.483


In [8]:
true_samples = true_dataset["text"].map(lambda x: x[:512])[:10]
fake_samples = fake_dataset["text"].map(lambda x: x[:512])[:10]

In [9]:
explain_text(fake_samples[0])

Explaining the following text: 
---------------------------------------------------------
Donald Trump just couldn t wish all Americans a Happy New Year and leave it at that. Instead, he had to give a shout out to his enemies, haters and  the very dishonest fake news media.  The former reality show star had just one job to do and he couldn t do it. As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year,  President Angry Pants tweeted.  2018 will be a great year for
--------------------------------------------------------
Prediction: [[{'label': 'LABEL_0', 'score': 0.9999857544898987}, {'label': 'LABEL_1', 'score': 1.4213927897799294e-05}]]


  0%|          | 0/48 [00:00<?, ?it/s]

Partition explainer: 2it [00:10, 10.16s/it]               


,Donald Trump just,couldn t,wish all,Americans a Happy New Year and leave it at that.,"Instead, he had to give a shout out to his enemies, haters and the very dishonest fake news media.",The former reality show star had just,one job to do,and he couldn t do it.,"As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters,","and even the very dishonest Fake News Media, a Happy and Healthy New Year, President Angry Pants tweeted. 2018 will be a great year for"
LABEL_0,0.072,4.072,0.908,0.107,-0.098,-0.096,0.865,4.029,0.023,0.221
LABEL_1,-0.072,-4.072,-0.908,-0.108,0.099,0.096,-0.865,-4.029,-0.023,-0.22
